# P1: Solve the OpenAI Gym [Taxi V3](https://gym.openai.com/envs/Taxi-v3/) Environment
---

## Introduction
[OpenAI Gym](https://gym.openai.com/docs/) is a framework that provides RL environments of varying complexity with the same standard API making it easy to develop and benchmark RL algorithms. The [Taxi-V3](https://gym.openai.com/envs/Taxi-v3/) environmnet present a simple, text environment where actions and state (observations) are both discrete. 

In [1]:
import gym

The `gym.make()` API can be used to spawn any of the available environments by passing its full name.

In [2]:
taxi = gym.make('Taxi-v3')

The Taxi environment has 500 states and 6 possible actions.

In [3]:
taxi.action_space

Discrete(6)

In [4]:
taxi.observation_space

Discrete(500)

The task and reward structure are described in the [documentation](https://github.com/openai/gym/blob/a5a6ae6bc0a5cfc0ff1ce9be723d59593c165022/gym/envs/toy_text/taxi.py#L25)

In [5]:
taxi.reset()
taxi.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



## Submission
- Submit your solution as a Jupyter notebook. 
- Ensure that all cells in the notebook have been executed and the output is showing
- Ensure that your solution consistently reaches the average cumulative reward defined in the rubric (link below)
- Post your solution on Github and share the link to your commit as a direct message in Slack

# **1**
**Describe the methods and variables in the class DiscreteEnv which is the parent class of the Taxi V3 class.**

**Variables:**

nS: number of states

nA: number of actions

action_space: The Space object corresponding to valid actions

observation_space: The Space object corresponding to valid observations

s: current state (observation)

lastaction: action taken in the last step

P: A dictionary defining transitions. For each key (state s and action a), the value is a list of tuples. Each tuple defines the probability of reaching a possible next state (from given state and action), the specified next state, the associated reward and whether the episode is done: 
P[s][a] == [(probability, nextstate, reward, done), ...]

isd: An array of length nS defining the initial state distribution 

**Methods:**

__init__(self, nS, nA, P, isd):

>   Initialize the DiscreteEnv object with the given input information (number of states, number of actions, transitions and initial state distribution), and draws a random initial state. 

seed(self, seed):

>   Sets the seed for this env's random number generator(s).

>   Returns:
*   the list of seeds used in this env's random number generators.

step(self, action): 

> Run one timestep of the discrete environment's dynamics according to the transition table P.

> Returns:
*   observation (object): agent's observation of the current environment, i.e., the new state
*   reward (float) : amount of reward returned after previous action
*   done (bool): whether the episode has ended
*   info (dict): contains auxiliary information, in this case the probability p associated with this new state

reset(self):

>   Run one timestep of the environment's dynamics.

>   Returns:
*   observation (object): the initial observation.


# **2**
**Describe the methods and variables in the Taxi V3 class.**

**Variables:**

desc: 2-dimensional array describing the map of the taxi environment.

locs: A list of valid passenger pick-up and drop-off locations in terms of row and column indices in the map. 

**Methods**

__init__(self):

Initialize the Taxi V3 object, including its class variables. The dictionary P defining the transitions (probabilities, next states, rewards, done) from all states and all different actions are defined. The initial state distribution is computed. The init function of the parent class is called to initialize the class variables in DiscreteEnv.

encode(self, taxi_row, taxi_col, pass_loc, dest_idx):

> Encode taxi locations （taxi_row, taxi_col）, passenger location (pass_loc), and destination index (dest_idx) into the corresponding state.

> Returns: 
* State 

decode(self, i):

> Decode the taxi locations, passenger location and destination location from the corresponding state.

> Returns:
* Taxi location (taxi_row, taxi_col), passenger location(pass_idx), and destination index (dest_idx)

render(self, mode):

>   Renders the environment.









# **3** 
**Describe the Taxi V3 environment, its actions, states, reward structure and the rationale behind such a reward structure.**

Taxi V3 environment has 500 discrete states, corresponding to 25 taxi locations on the 5x5 map, 5 passenger locations (4 possible pickup locations identified as R, G, B and Y on the map, plus passenger in the taxi), and 4 destination locations (R, G, B and Y). 

At each step the taxi can take one of 6 different actions: 4 actions corresponding to moving 1 unit in east, west, north, south directions, pick up passenger, and drop off passenger.

The reward structure is as follows: 
* Delivering the passenger at proper destination: +20,
* Executing "pickup" and "drop-off" actions illegally: -10,
* All other cases: -1.

Since the goal is to deliver the passenger at the right destination, it makes sense to have a large reward (20) when this task is accomplished.

Meanwhile, it is desirable to take as fewer steps as possible to deliver the passenger, so for each additional step taken to accomplish the task, a -1 reward is incurred. 

The illegal pickup happens if the passenger is not at the taxi location at the action of pickup. In this case, the pickup action is wasted and implies that the taxi will take more time to reach the passenger location. The -10 reward is for the algorithm to learn to move to passenger location and only pick up at the passenger location. 

The illegal drop-off happens if the passenger is not in the car, or the taxi location is not at one of the 4 possible destination points at the action of drop-off. The dropoff action is wasted in this case, and the taxi will take more time to pick up the passenger (if needed) and go the desired destination. The -10 reward is for the algorithm to learn that dropoff should only happens when the passenger is already picked up and the taxi has moved to one of the possible destinations. Note that dropping-off at possible destination points other than the desired one only incurs -1 reward, possibly because it is legal to drop off there and the taxi can make up by picking up the passenger again at this point in the next step. 





# **4**
**Train an algorithm to achieve a 100-episode average reward with a 5th percentile of 7.2 or higher and a 95th percentile of 8.2 or higher on the last 1000 episodes.**

In [6]:
def epsilon_greedy_action_from_Q(Q, state, epsilon):
    actions = Q.columns
    action_probs = np.asarray([epsilon/len(actions)]*len(actions),dtype=np.float)
    
    greedy_action_index = np.argmax(Q.loc[state].values)
    action_probs[greedy_action_index] += 1-epsilon

    epsilon_greedy_action = np.random.choice(Q.columns,p=action_probs)
    
    return epsilon_greedy_action

In [21]:
#from tqdm import tqdm 
import numpy as np
import pandas as pd
Q = pd.DataFrame.from_dict({s:{a:0 for a in range(taxi.nA)} for s in range(taxi.nS)}, orient='index')

HYPER_PARAMS = {'gamma':0.9}

n_episodes = 5000
max_episode_len = 100
epsilon = 1
min_epsilon = 0.01
epsilon_decay = 0.99
alpha = 0.1

rewards = np.zeros(n_episodes)

for i in range(n_episodes):  
    taxi.reset()
    s0 = taxi.s
    done = False
    
    episode_reward = 0

    for step in range(max_episode_len):
        a0 = epsilon_greedy_action_from_Q(Q,s0,epsilon)
        out  = taxi.step(a0)
        s1 = out[0]
        reward = out[1]
        done = out[2]
        
        Q.loc[s0,a0] += alpha*(reward + HYPER_PARAMS['gamma']*Q.loc[s1].max() - Q.loc[s0,a0])
        episode_reward += reward
        s0 = s1

        assert (reward!=-10) or (i<n_episodes-1000)

        if done:
          break

    if i%100 == 0:
      print ("reward for episode {}: {}".format(i, episode_reward))
  
    epsilon *= epsilon_decay
    epsilon = max(epsilon,min_epsilon) if i<n_episodes-1000 else 0  

    rewards[i] = episode_reward
        

reward for episode 0: -397
reward for episode 100: -271
reward for episode 200: -118
reward for episode 300: -109
reward for episode 400: -100
reward for episode 500: -74
reward for episode 600: -2
reward for episode 700: -95
reward for episode 800: -73
reward for episode 900: -73
reward for episode 1000: 14
reward for episode 1100: -11
reward for episode 1200: -25
reward for episode 1300: 8
reward for episode 1400: 10
reward for episode 1500: -5
reward for episode 1600: 9
reward for episode 1700: 8
reward for episode 1800: -3
reward for episode 1900: 14
reward for episode 2000: 5
reward for episode 2100: -2
reward for episode 2200: 8
reward for episode 2300: 7
reward for episode 2400: 9
reward for episode 2500: 3
reward for episode 2600: 6
reward for episode 2700: 7
reward for episode 2800: 8
reward for episode 2900: 8
reward for episode 3000: 10
reward for episode 3100: 9
reward for episode 3200: 13
reward for episode 3300: 5
reward for episode 3400: 10
reward for episode 3500: 8
rew

In [20]:
windowed_rewards = np.convolve(rewards[-1000:], np.ones(100), 'valid')
np.quantile(windowed_rewards/100,[0.05, 0.95])


array([7.41, 8.53])

As shown above, the trained algorithm based on Q-learning achieves the 100-episode average reward with 5th percentile of 7.41 and 95th percentile of 8.53 over the last 1000 episodes.

# 5
**The algorithm should be able to perform pick-ups and dropoffs with zero penalties over 1000 episodes.**

As shown above, the program will assert when an illegal pick-up or dropoff (defined as those with -10 reward) occurs in the last 1000 episodes. 
The program did not assert showing that there is no illegal pick-up or dropoffs in the last 1000 epislodes. 

# 6
**Document your solution including all hyper parameters and how those hyperparameters were selected.**

I solved the taxi problem with the basic reinforcement learning Algorithm: Q-learning algorithm. The Q-learning is an off-policy temporal-difference learning algorithm.  

At each step t, the action is chosen based on an epsilon-greedy approach
a_t = argmax_a Q(s_t,a). The ε-greedy algorithm takes the best (greedy) action with probability 1-ε, and with probability ε it takes a random action. Typically, the initial ε is large to encourage more exploration, and as learning proceeds the ε is gradually reduced to favor the greedy action (exploitation). 

After applying action A_t, we observe reward R_{t+1} and get into the next state s_{t+1}.

The action-value function is updated based on the following:

Q(s_t,a_t) <-- Q(s_t,a_t)+ α*(R_{t+1} + γ * max_a Q(s_{t+1},a) - Q(s_t,a_t)).

Note that unlike SARSA, at time t, Q-learning does not follow the current policy to pick the second action A_{t+1}, but instead just use the Q value associated with the greedy policy at t+1 for updating Q(s_t,a_t). 

The hyper parameters used in the taxi problem are as follows: 
* gamma = 0.9
* n_episodes = 5000
* max_episode_len = 100
* epsilon = 1  
* min_epsilon = 0.01
* epsilon_decay = 0.99
* alpha = 0.1

I started with a much smaller alpha value (0.001), and because of the large state space it takes very long for the algorithm to update the Q value and hence learning the optimal policy. By increasing the alpha value to 0.1, the convergence is much faster.

I also decreased the epsilon_decay rate from 0.999 to 0.99 to expedite the process of moving towards exploitation from exploration. This also helps with faster convergence.  

The maximum episode length was chosen to be 100, as more than that is contributing very little to the cumulative return and a waste of learning time. 

Finally, it was found that with the above hyper parameters, the algorithm is able to achieve the desired performance in last 1000 episodes with a total 5000 training episodes. Using more training episodes is not necessary. 

## Evaluation
The goal of the project is to get a certain average (cumulative) reward over 100 episodes. To pass the project, you must meet all the requirments in the project [rubric](https://github.com/KnowchowHQ/rl-in-action/blob/master/C1-RL-Intro/W3OH/P1-rubric.md)